# N3 — 00_ETL (Geração do dataset limpo)

Este notebook:
- **Baixa (se necessário) o dataset original diretamente do GitHub** (URL raw)
- Cria as pastas em `data/` automaticamente
- Realiza limpeza básica e engenharia de atributos
- Gera o dataset final para modelagem em `data/dataset_processado_N3/`

**Entrada (cache local):** `data/dataset_original/tb_mercadoimob.csv`  
**Fonte (GitHub raw):** `data/dataset_original/tb_mercadoimob.csv` no repositório `N3_Ciencia_Dados`  
**Saída:** `data/dataset_processado_N3/base_modelagem.csv`  
**Relatório ETL:** `data/dataset_processado_N3/etl_report.json`


In [ ]:
from __future__ import annotations

from pathlib import Path
import json
import re

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

def find_project_root(start: Path | None = None) -> Path:
    """Sobe diretórios até encontrar um marcador do projeto (requirements.txt ou pasta data)."""
    start = (start or Path.cwd()).resolve()
    for p in [start, *start.parents]:
        if (p / "requirements.txt").exists() or (p / "data").exists():
            return p
    return start

ROOT = find_project_root()

# URL RAW correta (sem /refs/heads/)
RAW_URL = "https://raw.githubusercontent.com/zilioalberto/N3_Ciencia_Dados/main/data/dataset_original/tb_mercadoimob.csv"

# Caminho local (cache)
RAW_PATH = ROOT / "data" / "dataset_original" / "tb_mercadoimob.csv"
RAW_PATH.parent.mkdir(parents=True, exist_ok=True)

# Saídas
OUT_DIR = ROOT / "data" / "dataset_processado_N3"
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_PATH = OUT_DIR / "base_modelagem.csv"
REPORT_PATH = OUT_DIR / "etl_report.json"

print("ROOT:", ROOT)
print("RAW_PATH:", RAW_PATH)
print("OUT_PATH:", OUT_PATH)


In [ ]:
# 1) Carregar dados (GitHub -> cache local)
import requests
from io import StringIO

def _detect_sep_from_header(text: str) -> str:
    header = (text.splitlines()[0] if text else "")
    return ";" if header.count(";") > header.count(",") else ","

def _read_csv_text(text: str) -> pd.DataFrame:
    sep = _detect_sep_from_header(text)
    return pd.read_csv(StringIO(text), sep=sep)

def load_csv_with_cache(url: str, local_path: Path) -> pd.DataFrame:
    if local_path.exists():
        print("✅ Lendo do cache local:", local_path)
        return pd.read_csv(local_path)

    print("⬇️ Baixando do GitHub:", url)
    r = requests.get(url, timeout=30)
    r.raise_for_status()

    df = _read_csv_text(r.text)

    print("💾 Salvando no cache local:", local_path)
    df.to_csv(local_path, index=False)

    return df

df_raw = load_csv_with_cache(RAW_URL, RAW_PATH)

print("Shape raw:", df_raw.shape)
display(df_raw.head(5))


In [ ]:
# 2) Funções utilitárias

def to_numeric_safe(s: pd.Series) -> pd.Series:
    """Converte para numérico aceitando formatos BR (1.234,56) e símbolos."""
    if pd.api.types.is_numeric_dtype(s):
        return s

    x = s.astype(str).str.strip()

    # remove espaços e textos como "R$"
    x = x.str.replace(r"\s+", "", regex=True)
    x = x.str.replace("R$", "", regex=False)

    # mantém apenas dígitos e separadores
    x = x.str.replace(r"[^0-9,\.\-]", "", regex=True)

    # casos:
    # - se tiver '.' e ',' -> assume '.' milhar e ',' decimal
    has_dot = x.str.contains(r"\.", regex=True)
    has_comma = x.str.contains(",", regex=False)

    x = x.where(~(has_dot & has_comma), x.str.replace(".", "", regex=False).str.replace(",", ".", regex=False))
    # - se tiver só ',' -> assume ',' decimal
    x = x.where(~(~has_dot & has_comma), x.str.replace(",", ".", regex=False))

    return pd.to_numeric(x, errors="coerce")

def normalize_text(s: pd.Series) -> pd.Series:
    return (
        s.fillna("")
         .astype(str)
         .str.replace(r"\s+", " ", regex=True)
         .str.strip()
         .str.lower()
    )

def to_binary_flag(s: pd.Series) -> pd.Series:
    """Normaliza colunas Sim/Não, True/False, 1/0 para 0/1."""
    if pd.api.types.is_bool_dtype(s):
        return s.astype("int8")
    if pd.api.types.is_numeric_dtype(s):
        return (s.fillna(0) != 0).astype("int8")

    x = normalize_text(s)
    true_set = {"1", "true", "t", "sim", "s", "yes", "y"}
    return x.isin(true_set).astype("int8")

# Padrões (texto -> flags)
vista_patterns = [
    r"\bvista\s*(para\s*o\s*)?mar\b",
    r"\bfrente\s*(para\s*o\s*)?mar\b",
    r"\bfrente\s*ao\s*mar\b",
    r"\bvista\s*mar\b",
]
mobiliado_patterns = [
    r"\bmobiliad[oa]\b",
    r"\bsemi\s*mobiliad[oa]\b",
    r"\bcom\s*m[óo]veis\b",
    r"\bcompleto\b",
]

vista_re = re.compile("|".join(vista_patterns), flags=re.IGNORECASE)
mobiliado_re = re.compile("|".join(mobiliado_patterns), flags=re.IGNORECASE)

def build_flags(df: pd.DataFrame) -> pd.DataFrame:
    """Cria colunas binárias com base em texto disponível."""
    df = df.copy()

    # tenta usar colunas de texto comuns (se existirem)
    text_cols_candidates = [
        "titulo", "descricao", "caracteristicas", "observacoes", "anuncio", "texto"
    ]
    text_cols = [c for c in text_cols_candidates if c in df.columns]

    if not text_cols:
        # fallback: tenta qualquer coluna object de tamanho razoável
        obj_cols = [c for c in df.columns if df[c].dtype == "object"]
        text_cols = obj_cols[:4]  # limita

    blob = ""
    for c in text_cols:
        blob = blob + " " + normalize_text(df[c])

    df["vista_mar_bin"] = blob.str.contains(vista_re, na=False).astype("int8")
    df["mobiliado_bin"] = blob.str.contains(mobiliado_re, na=False).astype("int8")
    return df

def clip_outliers_iqr(s: pd.Series, k: float = 1.5) -> pd.Series:
    """Clipa valores por IQR (não remove linhas)."""
    s = s.copy()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    if pd.isna(iqr) or iqr == 0:
        return s
    low = q1 - k * iqr
    high = q3 + k * iqr
    return s.clip(lower=low, upper=high)


In [ ]:
# 3) Seleção e limpeza básica

df = df_raw.copy()

# Colunas numéricas principais (se existirem)
num_cols = [
    "area", "valor", "iptu", "taxa_condominial",
    "num_quartos", "num_banheiros", "num_suites", "num_vagas_garagem", "num_andares",
]
for c in num_cols:
    if c in df.columns:
        df[c] = to_numeric_safe(df[c])

# Colunas binárias (se existirem)
bin_cols = ["imovel_lancamento", "bl_temporada"]
for c in bin_cols:
    if c in df.columns:
        df[c] = to_binary_flag(df[c])

# Flags a partir de texto
df = build_flags(df)

# Filtrar tipo de negócio (se existir)
if "tipo_negocio" in df.columns:
    tipo = normalize_text(df["tipo_negocio"])
    df = df[tipo.str.contains("venda", na=False)].copy()

# Garantir area positiva (se existir)
if "area" in df.columns:
    df = df[df["area"].notna() & (df["area"] > 0)].copy()

# Definir a variável alvo (preco_m2)
if "preco_m2" in df.columns:
    df["preco_m2"] = to_numeric_safe(df["preco_m2"])
elif "valor" in df.columns and "area" in df.columns:
    df["preco_m2"] = df["valor"] / df["area"]
elif "valor_m2" in df.columns:
    df["preco_m2"] = to_numeric_safe(df["valor_m2"])
else:
    raise KeyError("Não encontrei colunas para calcular 'preco_m2' (ex.: valor+area, preco_m2 ou valor_m2).")

# Remover/prevenir valores inválidos
df = df[df["preco_m2"].notna() & (df["preco_m2"] > 0)].copy()

# Clipping de outliers
df["preco_m2"] = clip_outliers_iqr(df["preco_m2"], k=1.5)

print("Shape após filtros:", df.shape)
display(df.head(5))


In [ ]:
# 4) Dataset final para modelagem

target_col = "preco_m2"

candidate_features = [
    # numéricas
    "area", "iptu", "taxa_condominial",
    "num_quartos", "num_banheiros", "num_suites", "num_vagas_garagem", "num_andares",
    # categóricas
    "tipo_imovel", "estado_construcao", "fonte",
    # binárias
    "imovel_lancamento", "bl_temporada",
    # texto->flags
    "vista_mar_bin", "mobiliado_bin",
]

feature_cols = [c for c in candidate_features if c in df.columns]

df_model = df[feature_cols + [target_col]].copy()

# Ajustes finais de tipos
for c in feature_cols:
    if c in ["tipo_imovel", "estado_construcao", "fonte"]:
        df_model[c] = df_model[c].fillna("desconhecido").astype(str)
    elif c in ["vista_mar_bin", "mobiliado_bin", "imovel_lancamento", "bl_temporada"]:
        df_model[c] = to_binary_flag(df_model[c])
    else:
        df_model[c] = to_numeric_safe(df_model[c])

df_model[target_col] = to_numeric_safe(df_model[target_col])

# Salvar
df_model.to_csv(OUT_PATH, index=False)

report = {
    "raw_url": RAW_URL,
    "raw_path": str(RAW_PATH),
    "out_path": str(OUT_PATH),
    "raw_shape": list(df_raw.shape),
    "after_basic_filters_shape": list(df.shape),
    "model_shape": list(df_model.shape),
    "features": feature_cols,
    "target": target_col,
    "notes": [
        "O dataset foi filtrado para tipo_negocio==Venda (se a coluna existir).",
        "preco_m2 possui clipping por IQR para reduzir influência de outliers sem descartar linhas.",
        "O CSV é baixado do GitHub e cacheado em RAW_PATH quando não existir localmente.",
    ],
}
REPORT_PATH.write_text(json.dumps(report, ensure_ascii=False, indent=2), encoding="utf-8")

print("✅ Salvo:", OUT_PATH)
print("✅ Salvo:", REPORT_PATH)
display(df_model.head(5))
